# The GASP Geobarometer
**JAR625M - Week 6 - Practical 1**

*Simon Matthews (simonm@hi.is)*

First, we need to import the necessary python libraries:

In [ ]:
from thermoengine import model
import numpy as np
import matplotlib.pyplot as plt

Now we need to extract the properties of the endmember components and the solid solutions:

In [ ]:
db = model.Database()

In [ ]:
# Solid Solutions:
garnet = db.get_phase('Grt')
feldspar = db.get_phase('Fsp')

# Pure endmembers:
grossular = db.get_phase('Grs')
kyanite = db.get_phase('Ky')
sillimanite = db.get_phase('Sil')
andalusite = db.get_phase('And')
anorthite = db.get_phase('An')
quartz = db.get_phase('Qz')


## Step 1: Calculate P,T-lines for particular logK values

The chemical reaction for the GASP geobarometer is:

Grossular (garnet) + Aluminosilicate + Quartz = Anorthite (Plagioclase)

Ca$_3$Al$_2$Si$_3$O$_{12}$ + 2 Al$_2$SiO$_5$ + SiO$_2$ = 3 CaAl$_2$Si$_2$O$_8$

At Equilibrium:

$\Delta G_r(P,T) = - R T \ln(K) $

If the phases are all pure endmembers then $K=1$ and $\ln(K) = 0$, so $\Delta G_r = 0$.

For impure minerals, i.e., solid solutions, $K \neq 1$, so $\Delta G_r \neq 0$, but we can calculate $K$ from $\Delta G_r(P,T)$ using:

$ K = \exp\left(- \frac{\Delta G_r(P,T)}{RT}\right) $

If we have a grid of $\Delta G_r$ values, like you created in the practical last week, then we can map out values for $K$ across P-T space

Here we can define a function to calculate $\Delta G_r$:

In [ ]:
def calc_GASP_DGr(T, P):

    # First, find which Al2SiO5 polymorph is stable:
    G_kyanite = kyanite.gibbs_energy(T, P)
    G_sillimanite = sillimanite.gibbs_energy(T, P)
    G_andalusite = andalusite.gibbs_energy(T, P)

    # Find the minimum Al2SiO5 Gibbs energy:
    G_AlSi = np.nanmin(np.array([G_kyanite, G_sillimanite, G_andalusite]))

    DGr = (
        3 * anorthite.gibbs_energy(T, P)
        - quartz.gibbs_energy(T, P)
        - 2 * G_AlSi
        - grossular.gibbs_energy(T, P)
    )

    return DGr

Now we will run this across a pressure and temperature grid:

In [ ]:
p = np.linspace(
    1, # Start pressure (bars)
    20000, # End pressure (bars)
    100 # Number of pressures to calculate
)

t = np.linspace(
    100 + 273.15, # Start temperature (K)
    1200 + 273.15, # End temperature (K)
    100 # Number of temperatures to calculate
)

# Turn this into a grid:
tt, pp = np.meshgrid(t, p)

# Create something to store the results:
DGr_grid = np.zeros(np.shape(tt))

# Iterate through each P,T point:
for i in range(np.shape(tt)[0]):
    for j in range(np.shape(tt)[1]):
        DGr_grid[i,j] = calc_GASP_DGr(tt[i,j], pp[i,j])



We can make a plot similar to the ones you made last week:

In [ ]:
f, a = plt.subplots()

cf = a.contourf(tt - 273.15, pp / 1000, DGr_grid/1000, vmin=-150, vmax=150, cmap=plt.cm.seismic, levels=1001)

cbar = f.colorbar(cf)

cbar.set_label(r'$\Delta G_r$ (kJ mol$^{-1}$)')

a.set_xlabel('Temperature (˚C)')
a.set_ylabel('Pressure (kbar)')


plt.show()

The white line marks the point where the reaction between pure endmembers goes from favourable to unfavourable. It is also defines the set of pressures and temperatures where the pure products AND pure reactants can exist together in the rock.

Convert the $\Delta G_r$ grid into the equilibrium constant K:

In [ ]:
K_grid = np.exp( - DGr_grid / 8.3141 / tt )

We can have python find lines of constant value of the equilibrium constant. To start with we can look at the case of $K=1$:

In [ ]:
# Tell the code which values of K to plot lines for:
K_contours = [1]

f, a = plt.subplots()

cf = a.contourf(tt - 273.15, pp / 1000, DGr_grid/1000, vmin=-150, vmax=150, cmap=plt.cm.seismic, levels=1001)

a.contour(tt - 273.15, pp / 1000, K_grid, levels=K_contours, colors='k')


cbar = f.colorbar(cf)

cbar.set_label(r'$\Delta G_r$ (kJ mol$^{-1}$)')

a.set_xlabel('Temperature (˚C)')
a.set_ylabel('Pressure (kbar)')


plt.show()

The expression for the equilibrium constant is:

$ K = \frac{a_\textrm{grossular}^\textrm{garnet} (a_\textrm{kyanite})^2 a_\textrm{quartz}}{(a_\textrm{anorthite}^\textrm{plagioclase})^3} $

Since kyanite and quartz are pure phases their activities are always 1, so it simplifies to:

$ K = \frac{a_\textrm{grossular}^\textrm{garnet}}{(a_\textrm{anorthite}^\textrm{plagioclase})^3} $

But if the endmembers are also pure, then both the top and bottom of the fraction will be 1, so $K=1$, $\ln(K) = 0$ and $\Delta G_r = 0$, so it falls at exactly the point the reaction changed from favourable to unfavourable.

A more interesting case is when the endmembers are not pure! We can plot lines that correspond to different values of K:


In [ ]:
# Tell the code which values of K to plot lines for:
K_contours = [ 0.001, 0.01, 0.1, 1, 10, 100, 1000]

f, a = plt.subplots()


cs = a.contour(tt - 273.15, pp / 1000, K_grid, levels=K_contours, colors='k',)

a.clabel(cs, fmt="K = %.3f")

a.set_xlabel('Temperature (˚C)')
a.set_ylabel('Pressure (kbar)')


plt.show()

## Part 2: Calculate activities and K values

The `garnet` object has an activity model built in, which we need in order to convert mole fractions into activities. We can get a feeling for how much of a difference this will make by making a plot of mole fraction vs activity. Here we will allow the garnet composition to vary between 100% almandine and 100% grossular:

In [ ]:
xgr = np.linspace(0, 1, 100)
xal = 1-xgr
agr = np.zeros(np.shape(xgr))
aal = np.zeros(np.shape(xgr))

t = 1200+273.15
p = 5000.0

for i in range(len(xgr)):
    [[aal[i], agr[i], _x]] = garnet.activity(t, p, mol=np.array([xal[i], xgr[i], 0]))

f, a = plt.subplots()

a.plot(xgr, agr, label='grossular')
a.plot(xgr, aal, label='almandine')

a.plot([0,1], [0,1], c='k', ls='--', lw=1)
a.plot([0,1], [1,0], c='k', ls='--', lw=1)

a.set_xlim(0,1)
a.set_ylim(0,1)

a.legend()

a.set_xlabel('Mole fraction of grossular in garnet')
a.set_ylabel('Activity of almandine or grossular')


plt.show()


You can see that calculating activities (rather than just assuming ideal mixing) will make a big difference to the results

You can play around with this by changing the pressure, temperature, and the mole fractions of almandine, grossular, and pyrope here:

In [ ]:

act = garnet.activity(
            1000 + 273.15, # Temperature (K)
            10000, # Pressure (bar)
            mol = np.array([
                0.3, # Almandine
                0.2, # Grossular
                0.5 # Pyrope
                ])
        )

enames = garnet.endmember_names

print("Activities of endmembers:")

for i in range(3):
    print("{0: <10}: {1:.4f} ".format(enames[i], act[0][i]))


In a gneiss containing an assemblage of quartz + plagioclase + alkali feldspar + garnet + biotite + kyanite + sillimanite + other accessory phases, there are zoned garnet and feldspar crystals. Below you are given the compositions measured in the both the cores and rims of the crystals.

| Oxide (wt%) | Garnet Core | Garnet Rim |
| ----------- | ----------- | ---------- |
| SiO~2~ | 37.9 | 37.4 |
| Al~2~O~3~ | 21.6 | 21.5 |
| FeO | 30.7 | 33.2 |
| MnO | 1.6 | 1.9 |
| MgO | 3.6 | 3.7 |
| CaO | 5.2 | 1.9 |

and...

| Oxide (wt%) | Plag Core | Plag Rim |
| ----------- | --------- | -------- |
| SiO~2~ | 59.4 | 58.8 |
| Al~2~O~3~ | 25.1 | 25.8 |
| Fe~2~O~3~ | 0.0 | 0.1 |
| CaO | 7.1 | 7.1 |
| Na~2~O | 7.6 | 7.4 |
| K~2~O | 0.2 | 0.5 |

You will estimate their pressure of formation by calculating the equilibrium constant for:
1. The garnet core - feldspar core
2. The garnet rim - feldspar rim

The first step towards calculating the activities of grossular and anorthite is to convert these compositions into moles of elements:
1. Divide the oxide wt% by the molecular mass of the oxide, e.g., for garnet core SiO2: 37.9 / 60.08 = 0.631
2. Multiply this number by the number of moles of the element in the oxide formula. SiO2 has 1 mole of Si, so 0.631 * 1 = 0.631
3. Repeat for each oxide (you do not need to calculate the moles of oxygen)


You can do this by hand, using excel, or in this notebook using python

In [ ]:
# Your code here?

You now need to convert the moles of elements into mole fractions of each solid solution endmember. For garnet you need mole fractions of almandine, pyrope, and grossular. For feldspar you need mole fractions of anorthite, albite and K-feldspar.
1. Find the formula for the endmember. E.g., grossular is Ca~3~Al~2~Si~3~O~12~.
2. There are 3 moles of Ca in the grossular formula. If you have 0.3 moles of Ca, this will give 0.1 moles of grossular. Why not use Al or Si? Because there is Al and Si in each of the garnet endmembers.
3. Repeat for each endmember.
4. Add up the moles of each endmember you calculated (e.g., 0.1 grossular + 0.2 almandine + 0.15 pyrope = 0.45)
5. Divide each endmember by the total, e.g., for grossular 0.1 / 0.45 = 0.22. 
6. They should now all add up to 1. Check to make sure they do!

You can do this in the notebook, on paper, or in Excel.

In [ ]:
# Your code here?

You should now have moles of endmembers for both garnet and feldspar in the rims and cores.

The next step is to convert them into activities. The activity depends on the temperature and pressure. A thermometer applied to the rocks has given a temperature of 500˚C. We don't know the pressure yet, but you can assume a value of 10 kbar (it is not very sensitive to pressure)

Use the code below to do the conversion for garnet:

In [ ]:

act = garnet.activity(
            500 + 273.15, # Temperature (K)
            10000, # Pressure (bar)
            mol = np.array([
                0.3, # Almandine
                0.2, # Grossular
                0.5 # Pyrope
                ])
        )

enames = garnet.endmember_names

print("Activities of endmembers:")

for i in range(3):
    print("{0: <10}: {1:.4f} ".format(enames[i], act[0][i]))


And for the feldspar:

In [ ]:
act = feldspar.activity(
            500 + 273.15, # Temperature (K)
            10000, # Pressure (bar)
            mol = np.array([
                0.3, # Albite
                0.69, # Anorthite
                0.01 # K-spar (sanidine)
                ])
        )

enames = feldspar.endmember_names

print("Activities of endmembers:")

for i in range(3):
    print("{0: <10}: {1:.4f} ".format(enames[i], act[0][i]))

Now replace the K contour values with the equilibrium constants you have calculated:

In [ ]:
# Tell the code which values of K to plot lines for:
K_contours = [ 0.001, 0.01, 0.1, 1, 10, 100, 1000]

f, a = plt.subplots()


cs = a.contour(tt - 273.15, pp / 1000, K_grid, levels=K_contours, colors='k',)

a.clabel(cs, fmt="K = %.3f")

a.axvline(500, c='r')


a.set_xlabel('Temperature (˚C)')
a.set_ylabel('Pressure (kbar)')


plt.show()

You should have found that the cores equilibrated at a higher pressure than the rims- it looks like the garnet grew as the rock reached its peak metamorphic conditions and has started to decompresss.
